**Customer Segmentation with RFM**

In [1]:
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#df_ = pd.read_excel("datasets/online_retail_II.xlsx", sheet_name= "Year 2009-2010")
#df = df_.copy()

#df["TotalPrice"] = df["Quantity"] * df["Price"]
#df.dropna(inplace=True)


### Calculating RFM Metrics ###

#today_date = dt.datetime(2010, 12, 11)

#rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
#                                     'Invoice': lambda num: num.nunique(),
#                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

#rfm.columns = ['recency', 'frequency', 'monetary']


### Calculating RFM Scores ###

#rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])

#rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

#rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])


#rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
#                    rfm['frequency_score'].astype(str))


### Creating RFM Segments ###

#seg_map = {
#    r'[1-2][1-2]': 'hibernating',
#    r'[1-2][3-4]': 'at_Risk',
#    r'[1-2]5': 'cant_loose',
#    r'3[1-2]': 'about_to_sleep',
#    r'33': 'need_attention',
#    r'[3-4][4-5]': 'loyal_customers',
#    r'41': 'promising',
#    r'51': 'new_customers',
#    r'[4-5][2-3]': 'potential_loyalists',
#    r'5[4-5]': 'champions',
#}



#rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)



**Customer Lifetime Value**

In [2]:
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#df_ = pd.read_excel("datasets/online_retail_II.xlsx", sheet_name= "Year 2009-2010")
#df = df_.copy()

#df["TotalPrice"] = df["Quantity"] * df["Price"]
#df.dropna(inplace=True)

#cltv_c = df.groupby('Customer ID').agg({
#    'Invoice': lambda x: x.nunique(),
#    'Quantity': lambda x: x.sum(),
#    'TotalPrice': lambda x: x.sum()})

#cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']

#cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]

#cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]

#repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]

#churn_rate = 1 - repeat_rate

#cltv_c['profit_margin'] = cltv_c["total_price"] * 0.10

#cltv_c['customer_value'] = cltv_c["average_order_value"] * cltv_c["purchase_frequency"]

#cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]

#cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])
#cltv_c.sort_values(by="cltv", ascending=False).head()